In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# When importing the dataset for every country you have to change the read_csv twice, 
# path.basename and to_excel  in the last row. Example DEvideos --> USvideos

# load the data
COvideos = pd.read_csv('DEvideos.csv')
CO_category_id = pd.read_csv('DE_category_id.csv')
filename = os.path.basename('DEvideos.csv')
title = filename[:2]

# merge the data
CO_category_id = CO_category_id.rename(columns={'items__id': 'category_id'})
COdataset = pd.merge(CO_category_id, COvideos, on='category_id')
COdataset1 = pd.merge(CO_category_id, COvideos, on='category_id')
COdata = COdataset.loc[:, ['channel_title','items__snippet__title', 'category_id', 'trending_date', 
                           'views', 'likes', 'dislikes', 'comment_count', 'comments_disabled', 
                           'ratings_disabled', 'video_error_or_removed']]
COdata = COdata.rename(columns={'items__snippet__title': 'category_name'})

# group by category and count the number of entries
category_count = COdata.groupby(['category_name']).size()

# calculate the percentage of each category and filter out categories with less than 3% participation
category_percent = category_count / len(COdata)
category_percent_filtered = category_percent[category_percent >= 0.03]
category_percent_filtered['Other'] = category_percent[category_percent < 0.03].sum()

# plot the pie chart
plt.pie(category_percent_filtered, labels=category_percent_filtered.index, autopct='%1.1f%%')
plt.title('Video Categories in ' + title + ' By Videos Trending')
plt.show()

total_count = COdata['views'].sum()

# filter out categories with less than 3% of the total count
threshold = 0.03 * total_count
COcategorycount = COdata.groupby(['category_name']).sum()
COcategorycount = COcategorycount[COcategorycount['views'] >= threshold]

# group the remaining categories into an "Other" category
other_count = total_count - COcategorycount['views'].sum()
COcategorycount.loc['Other'] = other_count

# create the pie chart
fig, ax = plt.subplots()
ax.pie(COcategorycount['views'], labels=COcategorycount.index, autopct='%1.1f%%')
ax.set_title('Video Categories in ' + title + ' By Views')

plt.show()

#Creating a total row
COcategorized = COdata.groupby(['category_name'])[['views','likes','dislikes']].sum().reset_index()
grouped = COcategorized.groupby('category_name').sum()
total_row = pd.DataFrame({'category_name': 'Total',
                          'views': grouped['views'].sum(),
                          'likes': grouped['likes'].sum(),
                          'dislikes': grouped['dislikes'].sum()}, index=[0])       
COcategorized = pd.concat([COcategorized, total_row])

#Calculating the ratios
COcategorized['likes/(dislikes + likes)[%]'] = ((COcategorized['likes'] / (COcategorized['likes']  + COcategorized['dislikes']))*100).round(2)
COcategorized['dislikes/(dislikes + likes)[%]'] = ((COcategorized['dislikes'] / (COcategorized['likes']  + COcategorized['dislikes'])*100)).round(2)
COcategorized['likes/views[%]'] = (COcategorized['likes'] / COcategorized['views'] * 100).round(2)

#Plot the second pie chart
total_likes = COcategorized['likes'].sum()
total_dislikes = COcategorized['dislikes'].sum()
fig, ax = plt.subplots()
ax.pie([total_likes, total_dislikes], labels=['Likes', 'Dislikes'], autopct='%1.1f%%')
ax.set_title('Likes vs. Dislikes Ratio')
plt.show()

#Exporting the ratio calculations for every Country
COratios = COcategorized[['category_name','likes/(dislikes + likes)[%]','dislikes/(dislikes + likes)[%]','likes/views[%]']]
COratios.to_excel('DEratios.xlsx', sheet_name='Sheet1', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'DEvideos.csv'